# En este notebook se hace un primer análisis de random forest para 50 árboles de decisión con la representación vectorial contextual BERT

In [ ]:
# To visualize plots in the notebook
%matplotlib inline

import numpy as np
import pandas as pd # To read data tables from csv files
import seaborn as sns # To plot statistical graphics
import matplotlib.pyplot as plt # To plot the figures
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

import os
from termcolor import colored
import tqdm
import scipy
import gc
import joblib
import torch

# For plots and graphical results
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab

# That's default image size for this interactive session
pylab.rcParams['figure.figsize'] = 6, 4

### **1. Preparación de los datos de entrenamiento y de test**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import torch
from sklearn.model_selection import train_test_split

# Cargar los datos desde el archivo JSON
data_path = '/content/drive/My Drive/Colab Notebooks/proyecto/full_format_recipes.json'
recipes_df = pd.read_json(data_path)

# Usamos los embeddings generados con BERT
bert_embeddings = torch.load(
    '/content/drive/My Drive/Colab Notebooks/proyecto/bert_embeddings.pt',
    weights_only=True  # Para evitar la advertencia de seguridad
)

# Asegurarnos de que el target contenga solo las primeras 10,000 recetas
target = recipes_df['rating'].fillna(recipes_df['rating'].mean()).iloc[:10000]

# Ahora dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    bert_embeddings, target, test_size=0.2, random_state=42
)

# Imprimir las formas de los datos para verificar
print(f"Tamaño de X_train: {X_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}")
print(f"Tamaño de y_train: {y_train.shape}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tamaño de X_train: torch.Size([8000, 768])
Tamaño de X_test: torch.Size([2000, 768])
Tamaño de y_train: (8000,)


### **2. Entrenamiento y evaluación**

#### ***2.1 Modelo de regresión con técnica de librería Scikit Learn***

##### *2.1.3 Random Forest*

In [ ]:
#para random forest no hay que normalizar los datos

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Entrenar un modelo de regresión lineal simple con los embeddings de BERT
model = LinearRegression()
model.fit(X_train,y_train)

# Predecir y evaluar
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

Mean Squared Error (MSE): 1.8387
R^2 Score: -0.1182


In [ ]:
#en regresión lineal ya se observa que R² negativo indican que el modelo no está explicando la variabilidad de los datos de forma efectiva,
# y que, de hecho, es peor que un modelo trivial que predice la media constante. Esto sugiere que los embeddings de BERT no están proporcionando
# una representación suficientemente informativa o útil para predecir las calificaciones de las recetas directamente.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Selección de las mejores características usando SelectKBest
n_components = 500  # Número de características a seleccionar
#con 100 componentes MSE de 1.80, con 500 dimensiones empeora!!! 1.8104
k_best = SelectKBest(f_regression, k=n_components)

# Aplicar la selección de características a los datos
X_train_selected = k_best.fit_transform(X_train, y_train)
X_test_selected = k_best.transform(X_test)

# Entrenar el modelo con las mejores características seleccionadas
model_kbest = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)

# Ajustar el modelo
model_kbest.fit(X_train_selected, y_train)

# Hacer predicciones
y_pred_kbest = model_kbest.predict(X_test_selected)

# Evaluar el rendimiento
mse_kbest = mean_squared_error(y_test, y_pred_kbest)
r2_kbest = r2_score(y_test, y_pred_kbest)

print("Resultados con SelectKBest:")
print(f"Mean Squared Error (MSE): {mse_kbest:.4f}")
print(f"R^2 Score: {r2_kbest:.4f}")

Resultados con SelectKBest:
Mean Squared Error (MSE): 1.8124
R^2 Score: -0.1022


In [ ]:
#Un aumento en el MSE al aumentar el número de componentes seleccionados por SelectKBest de 100 a 500 podría ser una señal
#de que las dimensiones adicionales introducen ruido en lugar de información útil.